# Test run GLONET on IMTA server

***
## Pre

In [1]:
import sys, os

src_path = os.path.abspath(os.path.join(os.getcwd(), "../glonet_daily_forecast_data_orchestration/src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from get_inits import generate_initial_data
from glonet_forecast import create_forecast
from model import synchronize_model_locally
from s3_upload import save_bytes_to_s3

/Odyssey/private/j25lee/miniforge3/envs/glon/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EDITO_BUCKET_NAME="project-da-ml"
URL="https://minio.dive.edito.eu/project-glonet/public/glonet_1_4_daily_forecast/2025-06-24/GLONET_MOI_2025-06-25_2025-07-04.nc"

In [3]:
import torch

print("PyTorch version:      ", torch.__version__)
print("CUDA available:       ", torch.cuda.is_available())
print("CUDA version (build): ", torch.version.cuda)
print("cuDNN version:        ", torch.backends.cudnn.version())

PyTorch version:       2.7.1+cu126
CUDA available:        True
CUDA version (build):  12.6
cuDNN version:         90501


***
## Input and model

In [4]:
# Divid inital states array in 3 files
cif1, cif2, cif3 = generate_initial_data(bucket_name=EDITO_BUCKET_NAME, 
                                        forecast_netcdf_file_url=URL)

Initial file already exists: https://minio.dive.edito.eu/project-glonet/public/glonet_1_4_daily_forecast/2025-06-24/inits/in1.nc
Initial file already exists: https://minio.dive.edito.eu/project-glonet/public/glonet_1_4_daily_forecast/2025-06-24/inits/in2.nc
Initial file already exists: https://minio.dive.edito.eu/project-glonet/public/glonet_1_4_daily_forecast/2025-06-24/inits/in3.nc


In [ ]:
import boto3
s3 = boto3.client("s3",endpoint_url = 'https://'+'minio.dive.edito.eu',
                  aws_access_key_id= '3BJNRN2OR05B3FNRQ7L2', 
                  aws_secret_access_key= 'o2oPuJd+aOClMIuG+djC0FcWuUy3lXI9PRQYi2Tw', 
                  aws_session_token = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiIzQkpOUk4yT1IwNUIzRk5SUTdMMiIsImFjciI6IjAiLCJhbGxvd2VkLW9yaWdpbnMiOlsiKiJdLCJhdWQiOlsibWluaW8iLCJhY2NvdW50Il0sImF1dGhfdGltZSI6MTc1MTY0MzI5MCwiYXpwIjoib255eGlhLW1pbmlvIiwiZW1haWwiOiJqdW5nd29uLmxlZUBpbXQtYXRsYW50aXF1ZS5mciIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJleHAiOjE3NTIwNjM3NzEsImZhbWlseV9uYW1lIjoiTEVFIiwiZ2l2ZW5fbmFtZSI6Ikp1bmd3b24iLCJncm91cHMiOlsiRURJVE9fVVNFUiIsImRhLW1sIiwiZ2xvbmV0Il0sImlhdCI6MTc1MTk3NzM3MSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmRpdmUuZWRpdG8uZXUvYXV0aC9yZWFsbXMvZGF0YWxhYiIsImp0aSI6Ijc2YmYwMTQ5LTNhMmYtNDM1Zi05MTZhLTYxN2QwMDE2OWJlMiIsIm5hbWUiOiJKdW5nd29uIExFRSIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJqdW5nd29uIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImRlZmF1bHQtcm9sZXMtZGF0YWxhYiIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX0sIm1pbmlvIjp7InJvbGVzIjpbInN0c29ubHkiXX19LCJzY29wZSI6Im9wZW5pZCBlbWFpbCBwcm9maWxlIiwic2Vzc2lvbl9zdGF0ZSI6IjUzMzUwMDhkLWJhMmEtNGQ5Yy1hM2EzLWYwZTViZWRiODA1OSIsInNpZCI6IjUzMzUwMDhkLWJhMmEtNGQ5Yy1hM2EzLWYwZTViZWRiODA1OSIsInN1YiI6ImJhMWM5ZjdmLTkzMGQtNGNiOC05NWU5LWEyNjAzOTU4ZWVlMyIsInR5cCI6IkJlYXJlciJ9._SJnkwepdpgLu5p-RdeYpJg7F3t16jIowFcSgXN7262uSZate2v9tKZGQisHMWvnmPJMKcf2wvmowwLRCkaviw')

from pathlib import Path


def synchronize_model_locally(local_dir: str):
    sync_s3_to_local(
        "project-glonet", "glonet_1_4_model/20241112/model/", local_dir
    )


def sync_s3_to_local(bucket_name, remote_prefix, local_dir):
    s3_client = s3
    paginator = s3_client.get_paginator("list_objects_v2")
    local_dir = Path(local_dir)
    local_dir.mkdir(parents=True, exist_ok=True)

    print(f"Syncing {bucket_name}/{remote_prefix} in {local_dir}...")
    for page in paginator.paginate(Bucket=bucket_name, Prefix=remote_prefix):
        if "Contents" not in page:
            print(f"No files found in s3://{bucket_name}/{remote_prefix}")
            return

        for obj in page["Contents"]:
            s3_key = obj["Key"]
            local_path = local_dir / s3_key[len(remote_prefix) :]

            local_path.parent.mkdir(parents=True, exist_ok=True)

            if (
                not local_path.exists()
                or obj["LastModified"].timestamp() > local_path.stat().st_mtime
            ):
                s3_client.download_file(bucket_name, s3_key, str(local_path))
    print(f"Files {bucket_name}/{remote_prefix} synced in {local_dir}")

local_dir = "../TrainedWeights"
synchronize_model_locally(local_dir=local_dir)

Syncing project-glonet/glonet_1_4_model/20241112/model/ in ../TrainedWeights...
